In [ ]:
import yahoo_fin.stock_info as yf
import pandas as pd
from stage import fullPrint
import random
from datetime import datetime, timedelta
import numpy as np
nyseDf = pd.read_html('stockList.html')
nyseDf = pd.DataFrame(nyseDf[0][0])
nyseList = []
for row in nyseDf.iterrows():
    ticker = str(row[1]).split()[1]
    if ticker == "Symbol":
        continue
    nyseList.append(ticker)
allList = []
nasdaqList = yf.tickers_nasdaq()
allList = nasdaqList + nyseList
tenYearSum = [0 for i in range(7)]
curSum = [0 for i in range(7)]
j = 0
random.shuffle(allList)
listOfStocks = [[] for i in range(4)]


: 

In [ ]:
from sklearn.model_selection import train_test_split
nasdaqList = yf.tickers_nasdaq()
train, test = train_test_split(nasdaqList, test_size=0.3, random_state=0, shuffle=True)
# np.random.shuffle(train)
trainSet1, trainSet2, trainSet3  = np.array_split(train,3)
trainSets = [trainSet1, trainSet2, trainSet3]

In [ ]:
# for i in range(len(allList)):
#     listOfStocks[i%4].append(allList[i])
#     # try:
#     #     df = pd.read_pickle("stockData/nyseNasdaq/"+allList[i]+".pkl")
#     # except:
#     #     pass
#     # try:
#     #     # drift = df.index[0].weekday()
#     #     # start = pd.to_datetime('2018-02-12',format = '%Y-%m-%d')
#     #     # end = pd.to_datetime('2022-01-10',format = '%Y-%m-%d')
#     #     # start = start + timedelta(days= drift)
#     #     # end = end + timedelta(days=drift)
#     #     a = df.iat[0,0]
#     #     b = df.iat[-1,0]
#     #     length = len(df.index)/52
#     #     # print(j%7,": first:",df.at[pd.to_datetime('2012-01-09',format = '%Y-%m-%d'),'close'],"second:",df.at[pd.to_datetime('2022-01-10',format = '%Y-%m-%d'),'close'])
#     #     if a != 0 and length > 3:
#     #         curSum[j%4] += (b/a)**(1/length)
#     #         print((b/a)**(1/length))
#     #         j = j + 1
#     #     # print(j%7,curSum[j%7], tenYearSum[j%7])
        
#     # except:
#     #     print(allList[i],"did not exist then")

In [ ]:
def calculateGroupReturn(list):
    monthlyReturn = pd.read_pickle("transactionTemplate.pkl")
    monthlyReturn = monthlyReturn.resample('M').mean()
    monthlyReturn.index = monthlyReturn.index.strftime("%Y:%m")
    currentColumn = -1
    for symbol in list:
        if symbol in monthlyReturn.columns:
            continue
        try:
            df = pd.read_pickle("stockData/nyseNasdaq/"+symbol+".pkl")
        except:
            continue
        if not (df['close'] != 0).all():
            continue
        monthlyReturn[symbol] = 1
        currentColumn += 1
        monthCheck = df.index[0].strftime("%Y:%m")
        maxPrice = df.iat[0,0]
        initialPrice = df.iat[0,0]
        for index, element in df.iterrows():
            
            if (index.strftime("%Y:%m") != monthCheck):
                if (maxPrice / initialPrice >= 1.03):
                    rowNumber = monthlyReturn.index.get_loc(index.strftime("%Y:%m"))
                    monthlyReturn.iat[rowNumber, currentColumn] = (maxPrice / initialPrice)
                initialPrice = element[0]
                maxPrice = element[0]
                monthCheck = index.strftime("%Y:%m")
            if (element[0] > maxPrice): maxPrice = element[0]
    return monthlyReturn
            
            
            

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning) 
i = 0
listOfDf = []
for l in trainSets:
    listOfDf.append(calculateGroupReturn(l))
print(listOfDf)


In [ ]:
product = 1
growthOfStock = {}
index = 0
for l in trainSets:
    for ticker in l:
        if ticker in listOfDf[index].columns:
            growthOfStock[ticker] = listOfDf[index].product()
    index += 1

In [ ]:
from stage import fullPrint
print(growthOfStock)

In [ ]:
sum = [0,0,0]
index = 0
countOfFirst = 0
countOfStocks = 0
for l in trainSets:
    print(sum[index])
    for ticker in l:
        if ticker in growthOfStock:
            sum[index] += growthOfStock[ticker][1]
            if index == 0:
                countOfFirst += 1
            countOfStocks += 1
    print(sum[index])
    sum[index] = sum[index] * (countOfFirst/countOfStocks)
    print((countOfFirst/countOfStocks))
    index += 1
    countOfStocks = 0
print(sum)
            

In [ ]:
fullPrint(listOfDf['AMOT'])